In [ ]:
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import os


batch_size = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import ssl

# Create a new SSL context that doesn't verify certificates
unverified_context = ssl._create_unverified_context()
# Use this context when making your HTTPS requests

In [ ]:
#addddddddd

In [ ]:
# adddada asdsad 

In [ ]:
import torchvision
from torchvision import transforms

train_mnist = torchvision.datasets.OxfordIIITPet('data/train',  transform=None, download=True)
test_mnist = torchvision.datasets.OxfordIIITPet('data/test',  transform=None, download=True)

train_mnist_tensor = torchvision.datasets.OxfordIIITPet('data/train',  transform=transforms.Compose([transforms.ToTensor()]), download=True)
test_mnist_tensor = torchvision.datasets.OxfordIIITPet('data/test',  transform=transforms.Compose([transforms.ToTensor()]), download=True)

train_loader = torch.utils.data.DataLoader(
                                    dataset = train_mnist_tensor,
                                    batch_size = batch_size,
                                    shuffle = True                                    
                                    )

test_loader = torch.utils.data.DataLoader(
                                    dataset = test_mnist_tensor,
                                    batch_size = batch_size,
                                    shuffle = False                                  
                                    )


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)>